Control Flow Prediction -> Part 2: Suffix Training
===


Process Prediction that focuses only on the control flow perspective of processes

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from exp.control_flow_prediction import *

In [3]:
log=import_xes(untar_data(URLs.BPIC_2012)).events

In [4]:
cfl=ControlFlowList.from_df(log)
sd=SplitData.split_by_func(cfl, partial(random_splitter, p_valid=0.1))
proc_tok,proc_num = TokenizeProcessor(),NumericalizeProcessor()
ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])
ll

SplitData
Train: LabeledData
x: ControlFlowList (10673 items)
['9->10->12->4->4->16->13->17->14->15->5->4->5->5->5->5->5->19->7->5->7->21->22->24->23->7->3', '9->10->12->4->4->4->4->16->17->13->14->15->5->4->5->5->5->13->18->14->15->5->5->5->5->5->19->7->5->7->7->7->7->7->24->22->21->23->7->3'...]
y: ControlFlowList (10673 items)
['0', '0'...]

Valid: LabeledData
x: ControlFlowList (1299 items)
['9->10->11->3', '9->10->12->4->4->16->17->13->14->15->5->4->5->5->5->5->5->19->7->5->7->22->23->21->24->7->3'...]
y: ControlFlowList (1299 items)
['0', '0'...]

Test: LabeledData
x: ControlFlowList (1115 items)
['9->10->8->8->12->4->8->4->4->4->4->4->11->4->3', '9->10->12->4->4->4->4->16->13->17->14->15->5->4->5->5->5->19->7->5->7->11->25->7->3'...]
y: ControlFlowList (1115 items)
['0', '0'...]


In [9]:


train_ds=Dataset(*_prepare_suffix_prediction(ll.train))
valid_ds=Dataset(*_prepare_suffix_prediction(ll.valid))

bs=64
train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds[t][0]), bs=bs//2)
valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds[t][0]))

train_dl = DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate_sp,drop_last=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, sampler=valid_sampler, collate_fn=pad_collate_sp,drop_last=True)

data=DataBunch(train_dl,valid_dl)

In [109]:
train_ds.x

[tensor([9]),
 tensor([ 9, 10]),
 tensor([ 9, 10, 12]),
 tensor([ 9, 10, 12,  4]),
 tensor([ 9, 10, 12,  4,  4]),
 tensor([ 9, 10, 12,  4,  4, 16]),
 tensor([ 9, 10, 12,  4,  4, 16, 13]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5,  5]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5,  5,  5]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5,  5,  5,  5]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5,  5,  5,  5,  5]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5,  5,  5,  5,  5, 19]),
 tensor([ 9, 10, 12,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5,  5,  5,  5,  5, 19,
          7]),
 tensor([ 9, 10, 12,  4

In [ ]:
data=_prepare_suffix_prediction(ll.train)

In [110]:
#export
def _prepare_suffix_prediction(ds, test=False):
    x,y=[],[]
    
    for o in ds.x.items:
        if len(o)<=1: continue
        for i in range(1,len(o)-1):
            x.append(tensor(o[:i]))
            y.append(tensor(o[i:]))
    return x,y

def suffix_data_set(data):
    return Dataset(*_prepare_suffix_prediction(data))    

def get_suffix_dls(train_ds, valid_ds, bs, **kwargs):
    train_ds,valid_ds=suffix_data_set(train_ds),suffix_data_set(valid_ds)
    train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds[t][0]), bs=bs//2)
    valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds[t][0]))
    return (DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate_sp,drop_last=True),
           DataLoader(valid_ds, batch_size=bs, sampler=valid_sampler, collate_fn=pad_collate_sp,drop_last=True))

def suffix_databunchify(ll, bs, **kwargs):
    return DataBunch(*get_suffix_dls(ll.train, ll.valid, bs, **kwargs))

In [111]:
data=suffix_databunchify(ll,64)

In [112]:
iter_dl = iter(data.train_dl)
xb,yb = next(iter_dl)

In [199]:
class Seq2SeqRNN(nn.Module):
    def __init__(self, itos, em_sz_enc, em_sz_dec, nh, out_sl, nl=2):
        super().__init__()
        self.nl,self.nh,self.out_sl = nl,nh,out_sl
        self.emb_enc = nn.Embedding(len(itos), em_sz_enc, padding_idx=1)
        self.emb_enc_drop = nn.Dropout(0.15)
        self.gru_enc = nn.GRU(em_sz_enc, nh, num_layers=nl, dropout=0.25)
        self.out_enc = nn.Linear(nh, em_sz_dec, bias=False)
                        
        self.emb_dec = nn.Embedding(len(itos), em_sz_dec, padding_idx=1)
        self.gru_dec = nn.GRU(em_sz_dec, em_sz_dec, num_layers=nl, dropout=0.1)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(em_sz_dec, len(itos))
        self.out.weight.data = self.emb_dec.weight.data
        
    def forward(self, inp):
        sl,bs = inp.size()
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, h = self.gru_enc(emb, h)
        h = self.out_enc(h)

        dec_inp = nn.Parameter(torch.zeros(bs)).long()
        res = []
        for i in range(self.out_sl):
            emb = self.emb_dec(dec_inp).unsqueeze(0)
            outp, h = self.gru_dec(emb, h)
            outp = self.out(self.out_drop(outp[0]))
            res.append(outp)
            dec_inp = nn.Parameter(outp.data.max(1)[1].float()).long()
            if (dec_inp==1).all(): break
        return torch.stack(res)
    
    def initHidden(self, bs): 
        return nn.Parameter(torch.zeros(self.nl, bs, self.nh))
    

In [200]:
def seq2seq_loss(input, target):
    bs,sl = target.size()
    bs_in,sl_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:,:sl,:]
    return F.cross_entropy(input.reshape(-1,nc), target.view(-1))#, ignore_index=1)

In [201]:
len(proc_num.vocab)

28

In [202]:
yb.shape

torch.Size([64, 115])

In [203]:
len(yb[0])

115

In [204]:
xb.shape

torch.Size([64, 174])

In [205]:
model=Seq2SeqRNN(proc_num.vocab,10,10,8,64)

In [206]:
pred=model(xb);pred.shape

torch.Size([64, 174, 28])

In [207]:
yb.shape

torch.Size([64, 115])

In [208]:
seq2seq_loss(pred,yb)

tensor(4.4900, grad_fn=<NllLossBackward>)

In [209]:
from torch.autograd import Variable



In [210]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressBarCallback]

In [211]:
learn = Learner(Seq2SeqRNN(proc_num.vocab,10,10,8,64).cuda(), data, seq2seq_loss, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [212]:
learn.fit(50)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time


RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu